In [1]:
import cv2
import os
import seaborn as sns
import json
import multiprocessing as mp
from tqdm import tqdm
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

sns.set_style("whitegrid", {'axes.grid' : False})

In [3]:
CONFIG = {
    "path_data":"/opt/ml/dataset",
    "info_data_train":"train.json",  # images, annotations
    "info_data_test":"test.json",
}

In [4]:
with open(os.path.join(CONFIG['path_data'], CONFIG['info_data_train'])) as js:
    json_train = json.load(js)

In [6]:
info_train = json_train['images']
annotations_train = json_train['annotations']
# x = number of annotations
# y = number of annotation's catagory
# s = sum of area
df_train = pd.DataFrame(info_train)
df_train = df_train.drop(columns = ['license', 'flickr_url', 'coco_url', 'date_captured'])
df_train['annotations'] = [[] for i in range(len(info_train))]
for an in annotations_train:
    df_train['annotations'].values[an['image_id']].append(an)
df_train['annotations_count'] = [len(ans) for ans in df_train['annotations'].values]
df_train['annotations_category_count'] = [len(set([an['category_id'] for an in ans])) for ans in df_train['annotations'].values]
df_train['annotations_sum_of_area'] = [int(sum([an['area'] for an in ans])) for ans in df_train['annotations'].values]
df_train.head()
print(df_train.describe())

        width  height           id  annotations_count  \
count  4883.0  4883.0  4883.000000        4883.000000   
mean   1024.0  1024.0  2441.000000           4.739709   
std       0.0     0.0  1409.745012           6.355155   
min    1024.0  1024.0     0.000000           1.000000   
25%    1024.0  1024.0  1220.500000           1.000000   
50%    1024.0  1024.0  2441.000000           2.000000   
75%    1024.0  1024.0  3661.500000           6.000000   
max    1024.0  1024.0  4882.000000          71.000000   

       annotations_category_count  annotations_sum_of_area  
count                 4883.000000             4.883000e+03  
mean                     1.934876             4.401477e+05  
std                      1.252878             3.899577e+05  
min                      1.000000             1.054000e+03  
25%                      1.000000             1.652355e+05  
50%                      1.000000             3.280820e+05  
75%                      3.000000             5.867090e+05 

In [7]:
annotations_train[0]

{'image_id': 0,
 'category_id': 0,
 'area': 257301.66,
 'bbox': [197.6, 193.7, 547.8, 469.7],
 'iscrowd': 0,
 'id': 0}

In [14]:
annotations_train[3]['bbox']

[722.3, 313.4, 274.3, 251.9]

In [18]:
w = annotations_train[3]['bbox'][0] - annotations_train[3]['bbox'][2]

In [19]:
h = annotations_train[3]['bbox'][1] - annotations_train[3]['bbox'][3]

In [22]:
w//h

7.0

In [55]:

anchor_ratio = []
for i in annotations_train:
    try:
        w = abs(i['bbox'][0] - i['bbox'][2])
        h = abs(i['bbox'][1] - i['bbox'][3])
        ratio = abs(round(w/h,1))
    except:
        print(i)
    anchor_ratio.append(ratio)


{'image_id': 674, 'category_id': 0, 'area': 10871.04, 'bbox': [282.2, 119.2, 91.2, 119.2], 'iscrowd': 0, 'id': 3156}
{'image_id': 1540, 'category_id': 9, 'area': 47981.96, 'bbox': [885.8, 348.2, 137.8, 348.2], 'iscrowd': 0, 'id': 7299}
{'image_id': 2320, 'category_id': 7, 'area': 81392.08, 'bbox': [364.7, 204.4, 398.2, 204.4], 'iscrowd': 0, 'id': 10940}
{'image_id': 3102, 'category_id': 0, 'area': 1972.08, 'bbox': [84.3, 49.8, 39.6, 49.8], 'iscrowd': 0, 'id': 14683}
{'image_id': 3231, 'category_id': 1, 'area': 12733.38, 'bbox': [300.2, 119.9, 106.2, 119.9], 'iscrowd': 0, 'id': 15321}
{'image_id': 4826, 'category_id': 7, 'area': 76209.12, 'bbox': [639.5, 318.6, 239.2, 318.6], 'iscrowd': 0, 'id': 22904}


In [56]:
check = {}
for i in anchor_ratio:
    if '{}'.format(i) in check:
        check['{}'.format(i)] += 1
    else:
        check['{}'.format(i)] = 1 
    

In [57]:
print(check)

{'1.3': 647, '0.3': 1209, '0.5': 1077, '7.3': 26, '0.2': 1238, '3.8': 104, '0.4': 1149, '0.1': 1270, '0.9': 887, '1.0': 927, '2.0': 316, '1.4': 579, '1.1': 746, '3.3': 116, '2.4': 215, '1.9': 364, '0.7': 1062, '1.6': 475, '17.3': 9, '0.8': 920, '1.5': 565, '10.8': 14, '2.3': 224, '3.9': 95, '8.6': 15, '5.4': 54, '2.8': 187, '3.6': 113, '8.2': 15, '11.4': 12, '33.1': 3, '1.7': 427, '1.2': 728, '9.1': 23, '6.2': 44, '3.1': 147, '10.1': 17, '3.0': 151, '0.0': 629, '0.6': 1096, '2.1': 291, '7.9': 21, '2.5': 194, '1.8': 401, '2.6': 191, '4.4': 72, '19.5': 4, '735.7': 1, '2.2': 261, '3.4': 121, '16.0': 6, '4.2': 65, '35.7': 2, '2.7': 169, '3.5': 127, '4.1': 87, '14.4': 7, '3.2': 137, '2.9': 139, '4.0': 96, '4.9': 52, '7.0': 32, '12.3': 10, '11.5': 8, '6.9': 31, '7.2': 16, '43.8': 4, '4.6': 56, '18.6': 4, '7.4': 23, '12.8': 5, '6.6': 41, '4.8': 71, '5.3': 51, '15.3': 7, '12.7': 9, '14.8': 10, '8.1': 20, '6.1': 36, '15.2': 5, '6.3': 30, '13.3': 5, '5.1': 45, '29.2': 1, '39.2': 1, '3.7': 104, '